In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from scipy.stats import ks_2samp
import plotly.graph_objects as go

In [2]:
def compute_statistics(train_feature, prod_feature):
    """
    Calcule les statistiques descriptives pour les données d'entraînement et de production.
    """
    train_mean = train_feature.mean()
    prod_mean = prod_feature.mean()
    train_std = train_feature.std()
    prod_std = prod_feature.std()
    train_min = train_feature.min()
    prod_min = prod_feature.min()
    train_max = train_feature.max()
    prod_max = prod_feature.max()

    return train_mean, prod_mean, train_std, prod_std, train_min, prod_min, train_max, prod_max

In [3]:
def calculate_differences(train_mean, prod_mean, train_std, prod_std, train_min, prod_min, train_max, prod_max):
    """
    Calcule les différences entre les statistiques des données d'entraînement et de production.
    """
    mean_diff = train_mean - prod_mean
    std_diff = train_std - prod_std
    min_diff = train_min - prod_min
    max_diff = train_max - prod_max

    return mean_diff, std_diff, min_diff, max_diff

In [4]:
def perform_ks_test(train_feature, prod_feature):
    """
    Effectue le test Kolmogorov-Smirnov entre les données d'entraînement et de production afin de revéler des difféences entres les deux distributions
    """
    ks_stat, p_value = ks_2samp(train_feature, prod_feature)
    return p_value

In [5]:
def plot_histogram(train_feature, prod_feature, feature):
    """
    Génère et affiche un histogramme comparatif entre les données d'entraînement et de production.
    """
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=train_feature, name="Train", nbinsx=30, histnorm='probability', opacity=0.7))
    fig.add_trace(go.Histogram(x=prod_feature, name="Production", nbinsx=30, histnorm='probability', opacity=0.7))
    fig.update_layout(
        title=f"Distribution Comparison for Continuous Feature: {feature}",
        xaxis_title=feature,
        yaxis_title="Density",
        barmode='overlay',
        legend=dict(title="Dataset")
    )
    fig.show()

In [6]:
def detect_continuous_data_drift(train_data, prod_data, feature_names, histogram=False):
    """
    Détecte le data drift pour des valeurs continues entre train_data et prod_data,
    avec des indicateurs statistiques supplémentaires et les différences entre chaque métrique.
    """
    results = []

    for feature in feature_names:
        if feature not in train_data.columns or feature not in prod_data.columns:
            continue

        # Sélectionner les données
        train_feature = train_data[feature].dropna()
        prod_feature = prod_data[feature].dropna()

        # Calcul des statistiques
        train_mean, prod_mean, train_std, prod_std, train_min, prod_min, train_max, prod_max = compute_statistics(train_feature, prod_feature)

        # Calcul des différences
        mean_diff, std_diff, min_diff, max_diff = calculate_differences(train_mean, prod_mean, train_std, prod_std, train_min, prod_min, train_max, prod_max)

        # Test Kolmogorov-Smirnov
        p_value = perform_ks_test(train_feature, prod_feature)

        # Calcul de la métrique de drift (différence absolue des moyennes)
        metric = np.abs(train_mean - prod_mean)

        # Enregistrer les résultats
        results.append({
            "Feature": feature,
            "P-Value": round(p_value, 4),
            "Drift Detected": "Yes" if p_value < 0.05 else "No",
            "Train Mean": round(train_mean, 2),
            "Prod Mean": round(prod_mean, 2),
            "Mean Difference": round(mean_diff, 2),
            "Train Std": round(train_std, 2),
            "Prod Std": round(prod_std, 2),
            "Std Difference": round(std_diff, 2),
            "Train Min": round(train_min, 2),
            "Prod Min": round(prod_min, 2),
            "Min Difference": round(min_diff, 2),
            "Train Max": round(train_max, 2),
            "Prod Max": round(prod_max, 2),
            "Max Difference": round(max_diff, 2),
            "Metric (Mean Difference)": round(metric, 2),
            "Drift Detected": "Yes" if p_value < 0.05 else "No"
        })

        if histogram:
          # Générer et afficher l'histogramme
          plot_histogram(train_feature, prod_feature, feature)

    return pd.DataFrame(results)

In [8]:
# Charger le dataset California Housing
data = fetch_california_housing(as_frame=True)
df = data['frame']

# Simuler des données de production en ajoutant un drift artificiel
train_data = df.sample(frac=0.7, random_state=42)  # 70% pour l'entraînement
prod_data = df.sample(frac=0.3, random_state=24)   # 30% pour la production

# Introduire un drift artificiel sur la variable 'MedInc'
prod_data['MedInc'] *= 1.2
prod_data['AveOccup'] *= 1.2

# Noms des features continues
feature_names = data['feature_names']  # Toutes les colonnes sont continues dans ce dataset

# Détecter le data drift
drift_results = detect_continuous_data_drift(train_data, prod_data, feature_names, histogram=True)


In [9]:
drift_results 

,Feature,P-Value,Drift Detected,Train Mean,Prod Mean,Mean Difference,Train Std,Prod Std,Std Difference,Train Min,Prod Min,Min Difference,Train Max,Prod Max,Max Difference,Metric (Mean Difference)
0,MedInc,0.0000,Yes,3.87,4.63,-0.76,1.90,2.27,-0.37,0.50,0.60,-0.10,15.00,18.00,-3.00,0.76
1,HouseAge,0.9986,No,28.69,28.66,0.03,12.60,12.58,0.02,1.00,1.00,0.00,52.00,52.00,0.00,0.03
2,AveRooms,0.9635,No,5.41,5.43,-0.02,2.28,2.19,0.09,0.85,0.85,0.00,132.53,52.69,79.84,0.02
3,AveBedrms,0.6097,No,1.09,1.10,-0.01,0.45,0.39,0.06,0.33,0.50,-0.17,34.07,10.27,23.80,0.01
4,Population,0.3161,No,1419.15,1433.58,-14.42,1112.12,1100.03,12.09,6.00,3.00,3.00,28566.00,16122.00,12444.00,14.42
5,AveOccup,0.0000,Yes,3.10,3.62,-0.52,11.68,7.70,3.99,1.06,0.83,0.23,1243.33,602.95,640.38,0.52
6,Latitude,0.9995,No,35.61,35.64,-0.02,2.13,2.15,-0.02,32.54,32.55,-0.01,41.95,41.88,0.07,0.02
7,Longitude,0.5393,No,-119.56,-119.57,0.01,2.00,2.02,-0.02,-124.35,-124.30,-0.05,-114.31,-114.47,0.16,0.01


Lorsqu'un **data drift** significatif est détecté, il est crucial de réagir pour garantir que le modèle reste performant et adapté aux nouvelles données. Voici plusieurs stratégies pour réentraîner le modèle en fonction du type et de la nature du drift détecté :

### **1. Réentraîner le modèle régulièrement (retrain périodique)**

- **Problème :** Le modèle peut devenir obsolète avec le temps, même sans drift immédiat.
- **Solution :** Planifier un réentraînement à intervalles réguliers (par exemple, tous les mois ou chaque trimestre). Cette approche permet de maintenir un modèle qui s’adapte aux tendances à long terme, en utilisant les nouvelles données de production.
  
  **Mise en œuvre :**
  - Créer un pipeline d'automatisation pour recueillir les nouvelles données et réentraîner le modèle périodiquement.
  - Utiliser des outils comme **MLflow**, **Airflow** ou **kestra** pour gérer et automatiser le réentraînement.

### **2. Réentraîner en cas de drift détecté (retrain basé sur le drift)**

- **Problème :** Le modèle pourrait être influencé par des variations non anticipées dans les données, comme un drift soudain.
- **Solution :** Lorsqu'un drift significatif est détecté (p-value < 0.05, par exemple), réentraîner le modèle en utilisant les nouvelles données.
  
  **Mise en œuvre :**
  - Intégrer une surveillance continue du modèle via des outils comme **Evidently AI** pour détecter le drift en temps réel.
  - Déclencher un processus de réentraînement automatique lorsque des seuils de drift sont dépassés. ou des **trigger systems**.

